In [1]:
import re
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 10000)
pd.options.display.max_rows = 5000
import warnings
warnings.filterwarnings("ignore")
import sys

#from similarityfunction import *
#from similarityfunctionGuise import *

from sklearn import metrics
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,roc_curve

import os
cwd = os.getcwd()
print("current directory", cwd)

current directory C:\Users\garvarun\Desktop\project


# Read actuals data

In [2]:
actual_df = pd.read_excel(r'C:\Users\garvarun\Desktop\project\Aflac Performance Dashboard -  EOM Feb 2023.xlsb', engine='pyxlsb')
actual_df = actual_df[["session_id", "agent", "evaluated_by", "section_1_answer_2", "section_1_answer_2_score", 
                       "section_1_question_2_comment", "call_type"]]

actual_df.loc[actual_df["section_1_answer_2"].isna(), "Actual"] = -1
actual_df.loc[actual_df["section_1_answer_2"] == "Yes", "Actual"] = 1
actual_df.loc[actual_df["section_1_answer_2"] == "No", "Actual"] = 0

print(actual_df.shape[0])
print(len(actual_df["session_id"].unique()))
print(actual_df["Actual"].value_counts())

8855
8827
 1.0    8847
 0.0       7
-1.0       1
Name: Actual, dtype: int64


In [3]:
import pickle

# Data loading and cleaning

In [4]:
data_raw = pickle.load(open(r'C:\Users\garvarun\Downloads\aflac_mongodb_data_5410.pkl', 'rb'))
#data_raw = data_raw[:10000]
print(data_raw.shape[0])
#clipboard_data = pd.read_clipboard(header=None)[0].tolist()

session_ids = ["KK5MPL8GG911R0E1AC5QUIOP7002D4F4",
               "KK5MPL8GG911R0E1AC5QUIOP7002F39G",
               "KK5MPL8GG911R0E1AC5QUIOP7002FVRK",
               "KK5MPL8GG911R0E1AC5QUIOP70067984",
               "0JI9SKPQQ971528JUUUSCORNTK004FPQ"]
#data_raw = data_raw[data_raw['session_id'].isin(clipboard_data)]

#data_raw = data_raw[data_raw['session_id'].isin(session_ids)]
print(data_raw.shape[0])
#595924
print(len(data_raw['session_id'].unique()))

1109648
1109648
5410


In [5]:
data_raw.columns

Index(['session_id', 'start_time', 'end_time', 'call_type', 'entity_id',
       'entity_email', 'time_stamp', 'chat_text', 'entity_type',
       'sentiment_score', 'overall_sentiment_score', 'sequence', 'agent_email',
       'duration', 'overall_duration', 'voice_time', 'client_time',
       'agent_time', 'silence_time', 'hold_time', 'num_transfers',
       'overlap_time', 'max_silence', 'sentiment', 'num_holds', 'last_speaker',
       'start_time1', 'end_time1'],
      dtype='object')

In [6]:
data_activity = pd.read_csv(cwd + "\\staging.tblactivity.csv")[["vchproductsold", "vchGenCallID", "iContactID"]]
data_raw = pd.merge(data_raw, data_activity, left_on=['session_id'], how='inner', right_on=['vchGenCallID'])


data_state = pd.read_csv(cwd + "\\staging.tblcontact.csv")[["vchFirstName", "vchLastName", "iContactID"]]
data_raw = pd.merge(data_raw, data_state, left_on=['iContactID'], how='inner', right_on=['iContactID'])


In [7]:
data_raw.columns


Index(['session_id', 'start_time', 'end_time', 'call_type', 'entity_id',
       'entity_email', 'time_stamp', 'chat_text', 'entity_type',
       'sentiment_score', 'overall_sentiment_score', 'sequence', 'agent_email',
       'duration', 'overall_duration', 'voice_time', 'client_time',
       'agent_time', 'silence_time', 'hold_time', 'num_transfers',
       'overlap_time', 'max_silence', 'sentiment', 'num_holds', 'last_speaker',
       'start_time1', 'end_time1', 'vchproductsold', 'vchGenCallID',
       'iContactID', 'vchFirstName', 'vchLastName'],
      dtype='object')

In [8]:
data_raw_filtered = data_raw.query("call_type == 'Outbound'").drop_duplicates(subset='session_id')
common_rows_count = len(pd.merge(data_raw_filtered, actual_df, on=['session_id', 'call_type']))

# Print the result
print("Number of rows with common session_id and call_type: ", common_rows_count)

Number of rows with common session_id and call_type:  162


In [9]:
pd.merge(data_raw.query("call_type == 'Outbound'"), actual_df.query("call_type == 'Outbound'"), on=['session_id', 'call_type'])

,session_id,start_time,end_time,call_type,entity_id,entity_email,time_stamp,chat_text,entity_type,sentiment_score,...,vchGenCallID,iContactID,vchFirstName,vchLastName,agent,evaluated_by,section_1_answer_2,section_1_answer_2_score,section_1_question_2_comment,Actual
0,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2023-02-06T15:44:33.000Z,2023-02-06T16:32:42.000Z,Outbound,20571029,leaha.santorum@sutherlandglobal.com,3.6250,Right of the morning.,agent,0.000000,...,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2927169.0,Leazhia,Hunter,Leah Santorum,Antony Varghese,Yes,3.0,NaN,1.0
1,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2023-02-06T15:44:33.000Z,2023-02-06T16:32:42.000Z,Outbound,20571029,leaha.santorum@sutherlandglobal.com,4.0625,Morning.,client,0.000000,...,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2927169.0,Leazhia,Hunter,Leah Santorum,Antony Varghese,Yes,3.0,NaN,1.0
2,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2023-02-06T15:44:33.000Z,2023-02-06T16:32:42.000Z,Outbound,20571029,leaha.santorum@sutherlandglobal.com,6.1875,This is Leah and Tori calling you from a flood insurance. You submitted an inquiry online at Flat Dot Com. Is this a good time for you?,agent,0.000000,...,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2927169.0,Leazhia,Hunter,Leah Santorum,Antony Varghese,Yes,3.0,NaN,1.0
3,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2023-02-06T15:44:33.000Z,2023-02-06T16:32:42.000Z,Outbound,20571029,leaha.santorum@sutherlandglobal.com,15.1875,Yeah.,client,0.000000,...,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2927169.0,Leazhia,Hunter,Leah Santorum,Antony Varghese,Yes,3.0,NaN,1.0
4,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2023-02-06T15:44:33.000Z,2023-02-06T16:32:42.000Z,Outbound,20571029,leaha.santorum@sutherlandglobal.com,15.9375,I like electioneering a call for quality assurance purposes. You're in. Excuse me. Where are you Indiana?,agent,0.000000,...,7JCCO8JK0T7UH4H1Q3FILGCK2S000LST,2927169.0,Leazhia,Hunter,Leah Santorum,Antony Varghese,Yes,3.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43878,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,583.1875,The.,client,0.000000,...,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2895053.0,Gustavo,Cervantes,Robert Hanmer,Antony Varghese,Yes,3.0,NaN,1.0
43879,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,591.1875,"Alright, sounds good. you have a good one, too.",client,0.956295,...,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2895053.0,Gustavo,Cervantes,Robert Hanmer,Antony Varghese,Yes,3.0,NaN,1.0
43880,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,594.0625,"Thank you take care. bye, bye.",agent,0.000000,...,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2895053.0,Gustavo,Cervantes,Robert Hanmer,Antony Varghese,Yes,3.0,NaN,1.0
43881,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,595.6875,"Alright, five back.",client,0.000000,...,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2895053.0,Gustavo,Cervantes,Robert Hanmer,Antony Varghese,Yes,3.0,NaN,1.0


# Voice message calls

In [10]:
voice_message_HC_baselines = [
    "please record your message", "when you've finished recording", "when you have finished recording", 
    "you may hang up or press one for more options", "to replay your message", "to delete and rerecord your message", 
    "to continue recording", "to cancel this message", "to send this message now", "press pound or hang up",
    "for delivery, options, press", "if you're satisfied with the message", "if you are satisfied with the message",
    "press one to listen to your message", "press two to erase and rerecord", "press three to continue recording where you left off",
    "to send a fax press six", "please leave your message for", "get your message either because you were not speaking",
    "because of a bad connection to disconnect press one to record your message", "are you still there? if you are satisfied with your message?",
    "forwarded to an automated voice messaging system", "automated voice messaging system",
    "mailbox is full and cannot accept any messages at this time", "nothing has been recorded", "record your message after the tone",
    "your message has been deleted", "your message has been sent", "to leave a callback number press five", 
    "we did not get your message", "either because you were not speaking, or because of a bad connection",
    "unfortunately, the system cannot process your entries", "person you called has a voice mail box",
    "to send an sms notification", 
    "if you're still interested in getting a quote with us, go ahead and give me a call back at",
    "if you are still interested in getting a quote with us, go ahead and give me a call back at",
    #Added 20 April
    "name and phone number after the beep", "at the tone when you are finished",
    "you have reached the voicemail", "record your message", "please leave a voice mail message", 
    "simply hang up to review", "please leave a brief message", "at the tone", "leave a message after the beep", 
    "is currently unavailable", "please leave a message", "number and reason for calling",
    "get back to you as soon as possible",
    ##Added 11 May
    "mailbox is full", "this message is for", "person you are trying to reach", "(voicemail|voice mail)(.*)(not been set up)" 
]

voimsg_sessions_df = data_raw[
    ((data_raw["chat_text"].str.contains("|".join(voice_message_HC_baselines), na=False, case=False)) 
     & (data_raw["sequence"] < 20)) |
    ((data_raw["chat_text"].str.contains("|".join(["voice message", "automatic voice", "(please)(.*)(leave)(.*)(number|name|message)", 
        "(subscriber)(.*)(available)", "(can't come|couldn't get to|come to)(.*)(phone)", ]), na=False, case=False)) 
     & (data_raw["sequence"] < 10))
][['session_id', 'sequence']].groupby("session_id").agg(list).reset_index()

voimsg_list = voimsg_sessions_df['session_id'].to_list()
len(voimsg_list) #, voimsg_list

0

In [11]:
wrongno_sessions_df = data_raw[
    ((data_raw["chat_text"].str.contains("|".join(["wrong number"]), na=False, case=False)) 
     & (data_raw["sequence"] < 10))][['session_id', 'sequence']].groupby("session_id").agg(list).reset_index()

wrongno_list = wrongno_sessions_df['session_id'].to_list()
len(wrongno_list) #, wrongno_list

0

# Call disconnection cases

In [12]:
from collections import defaultdict
import pandas as pd
import spacy

# Load the Spacy model
nlp = spacy.load("en_core_web_lg")

# Generate results for input data

In [13]:
def calculate_CustomerName(df, kpi_df, kpi_type):
    #print(df)
    d= defaultdict()
    #kpi_df=kpi_df[kpi_df.name==kpi_type]
    d['kpi_id'] = 1 #kpi_df.id.iloc[0]
    d['kpi_name'] = "Customer_Name" #kpi_df.name.iloc[0]
    
    
    res=[]
    sequence=[]
    person_names = []
    greetings = []

     
    customer_pings_df = df[(df["entity_type"] == "client")]
    customer_pings = customer_pings_df["chat_text"].tolist()
    agent_pings_df = df[(df["entity_type"] == "agent")]
    agent_pings = agent_pings_df["chat_text"].tolist() 
    if len(agent_pings) > 0:
        session_id = agent_pings_df.session_id.unique()[0]
      
        if ((session_id in voimsg_list) | (session_id in wrongno_list) | (df.shape[0] < 1)):
            res.append(('Call discon/ voimsg/ wrongNo', -1, []))
 
        else:
            customer_group = df[df['entity_type'] == 'client'].iloc[:15].sort_values(by=['time_stamp'])
            customer_text = ' '.join(customer_group['chat_text'])

            agent_group_ner = df[df['entity_type'] == 'agent'].iloc[:65].sort_values(by=['time_stamp'])
            agent_text_ner = ' '.join(agent_group_ner['chat_text'])

            agent_doc = nlp(agent_text_ner)
            agent_text = ' '.join([token.text for token in agent_doc])
            customer_doc = nlp(customer_text)
            customer_persons_ner = set([ent.text for ent in customer_doc.ents if ent.label_ == "PERSON" and 'nan' not in ent.text and not ent.text[0].islower()])
          
            agent_regex_miss = r'\b(?:Miss|miss|Mister|Mr|Sir|Madam|Mam|your lasting|firstinclass|speaking to|your name|Ms.|speak with|I need to speak with|speaking with|\.)\b'

            agent_group = df[df['entity_type'] == 'agent'].iloc[:65].sort_values(by=['time_stamp'])
            agent_text_miss = ' '.join(agent_group['chat_text'])
            agent_persons = set(re.findall(agent_regex_miss, agent_text_miss, re.IGNORECASE))
          
            agent_regex = r'\b(?:first|last|name)\b'
            agent_text_lines = agent_group['chat_text'].tolist()
          
            matches_list = list(set([matches for line in agent_text_lines for matches in re.findall(agent_regex, line, re.IGNORECASE)]))
           
            tokens = [token.text for token in agent_doc]
            indices = list(range(len(tokens)))

            person_names = [tokens[i] for i in indices if agent_doc[i].ent_type_ == "PERSON"]

            greetings = [tokens[i].lower() for i in indices if any(abs(indices.index(i) - indices.index(person_index)) <= 2 for person_index in indices if tokens[person_index] in person_names) and tokens[i].lower() in ["hey", "hi", "hello"]]
       
            if not customer_persons_ner or any(word in agent_text.split() for element in customer_persons_ner for word in element.split()):
                res.append(('NER', 1, []))
            elif agent_persons:
                res.append(('mister_misses', 1, []))
            elif len(matches_list) >= 2:
                res.append(('your_name', 1, []))
            elif greetings:
                res.append(('greetings', 1, []))
            elif any(word in agent_text.split() for element in df['vchFirstName'].tolist() for word in element.split()) or any(word in agent_text.split() for element in df['vchLastName'].tolist() for word in element.split()):
                res.append(('person_name', 1, []))
            else :
                res.append(('no_match', 0, []))
    else:
        res.append(('No Agent pings', -1, []))
    d['score']=res
    return d

In [14]:
import time
start = time.time()

####### Data loading and cleaning

#df = pd.read_csv(cwd + "\\nonsuccesssales_mar_2.csv")
#kpi_df = pd.read_excel(cwd + "\\kpi_df.xlsx")
kpi_type = "Customer_Name"

data_base = pd.merge(data_raw, actual_df[['session_id']], left_on=['session_id'], how='inner', right_on=['session_id'])

print("Evaluating on unique session ids ", len(data_base['session_id'].unique()))

out = data_base.groupby('session_id').apply(calculate_CustomerName, None, kpi_type)



k=pd.DataFrame(out,columns=['res']).reset_index()

k['agent_name']=k['res'].map(lambda x: x['score'][0][0])
k['kpi_result']=k['res'].map(lambda x: x['score'][0][1])
k['sequence']=k['res'].map(lambda x: x['score'][0][2]) 


print("KPI frequencies = ")
print(k['kpi_result'].value_counts())

end = time.time()
print("Time taken in min: ", round(((end - start)/60), 2) )

Evaluating on unique session ids  732
session_id 609U55O62L5GVFFI61EIGIGKI0000GFG
greetings []
session_id 609U55O62L5GVFFI61EIGIGKI0000LNJ
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000EFU
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000GBN
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000I5S
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000ILR
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000K43
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000LEG
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000LNV
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000LST
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000OPC
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000QB9
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000T0S
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000TT4
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000V6E
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S000VUC
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGC

greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00C26C
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00C3GT
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CABK
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CBBB
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CDS4
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CFK7
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CFMC
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CGCD
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CHIG
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CIMP
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CKGN
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00COEN
greetings ['hi']
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00COI2
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CQ9G
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CVJ1
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00CVT7
greetings []
session_id 7JCCO8JK0T7UH4H1Q3FILGCK2S00D34S
greetings []
se

greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST4003GJ4
greetings ['hi']
session_id L17LEQKG6H2A77JNOPQIFJ6ST4003J4D
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST4003OCE
greetings ['hi']
session_id L17LEQKG6H2A77JNOPQIFJ6ST4003Q71
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST4003QRH
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST4003R66
greetings ['hi']
session_id L17LEQKG6H2A77JNOPQIFJ6ST4003V8E
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40045A7
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40045BS
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40045J6
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40046EE
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40046JR
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40047ID
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40047K1
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST40047OR
greetings ['hey', 'hey']
session_id L17LEQKG6H2A77JNOPQIFJ6ST40048NQ
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST4004K7R
greetin

greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400B2F6
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400B9CH
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400B9F3
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400B9P0
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400BA48
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400BBFI
greetings ['hey']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400BD3A
greetings ['hey']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400BDMK
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400BDNO
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400BF5H
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400BFMP
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400CFUN
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400CJC1
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400CLTG
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400CM3B
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400CPSK
greetings ['hello']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400CRI9
greetings ['hi

greetings ['hi']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400OQQT
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400P2A1
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400P49N
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400P5KV
greetings ['hi']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400P5O1
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400P643
greetings ['hey']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400P6H6
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400P72H
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PISL
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PJ56
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PM2V
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PMEM
greetings ['hi']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PO2V
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PRAC
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PRI5
greetings []
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PSRN
greetings ['hi']
session_id L17LEQKG6H2A77JNOPQIFJ6ST400PVQ6
greetings 

In [17]:
k.to_csv('result_customer_name111.csv')

# Check results with actual data - Modify if required

In [16]:
result_df = pd.merge(k[["session_id", "agent_name", "kpi_result", "sequence"
    ]], actual_df, left_on=['session_id'], how='inner', right_on=['session_id'])

result_df.loc[result_df["Actual"] == result_df["kpi_result"], "Match"] = True
result_df.loc[result_df["Actual"] != result_df["kpi_result"], "Match"] = False

result_df["Actual"] = result_df["Actual"].astype(int)
print("actual value frequencies =", result_df[['Actual']].value_counts())
print("predicted value frequencies =", result_df[['kpi_result']].value_counts())
print("accuracy_score = ", metrics.accuracy_score(result_df["Actual"], result_df["kpi_result"]))
print("confusion_matrix = ") 
print(metrics.confusion_matrix(result_df["Actual"], result_df["kpi_result"]))

result_df[["session_id", "call_type", "agent", "evaluated_by", 
     "sequence",  
    "Actual", "kpi_result", "Match"]].to_excel(cwd + "//result_corrected_mismatch_actorchat_5365_22May.xlsx")

actual value frequencies = Actual
1         736
dtype: int64
predicted value frequencies = kpi_result
 1            658
-1             65
 0             13
dtype: int64
accuracy_score =  0.8940217391304348
confusion_matrix = 
[[  0   0   0]
 [  0   0   0]
 [ 65  13 658]]


In [36]:
data_raw[(data_raw['session_id'].isin(result_df[(result_df['Match'] == False)]['session_id'].to_list() )) ] \
        [['session_id', 'sequence', 'chat_text', 'entity_type']].sort_values(['session_id', 'sequence']) \
        .to_excel(cwd + "//TEMP_mismatch_actorchat_5365.xlsx")

In [39]:
result_df[['kpi_result']]

,kpi_result
0,1
1,1
2,1
3,1
4,1
...,...
5395,1
5396,1
5397,1
5398,1


In [19]:
#k = pd.read_excel(cwd + "//result_actor_chat_corrected_actorchat_merged_15_20_10May_eve.xlsx")

# Debug

In [20]:
data_vch = data_raw.fillna("Blank")[['session_id', 'vchproductsold']].drop_duplicates()
data_vch[data_vch.groupby(['session_id'])['vchproductsold'].transform('count') > 1]

KeyError: "['vchproductsold'] not in index"

In [ ]:
data_vch[~(data_vch['vchproductsold'] == "Blank")].shape[0]

In [ ]:
data_raw[(data_raw['session_id'].isin(result_df[(result_df['Actual'] == -1) & (result_df['Match'] == False)]['session_id'].to_list() )) 
        ][['session_id', 'vchproductsold']].drop_duplicates().sort_values(['session_id']) #.shape[0]


In [ ]:
data_raw[(data_raw['session_id'].isin(result_df[(result_df['Actual'] == 0) & (result_df['kpi_result'] == 1) 
                                & (result_df['Match'] == False) & (result_df['suitability_flag'] == False)]\
                                      ['session_id'].to_list() )) 
        ][['session_id', 'vchproductsold']].drop_duplicates().sort_values(['session_id']) #.shape[0]


In [ ]:
data_raw[(data_raw['session_id'].isin(result_df[(result_df['Actual'] == 0) & (result_df['kpi_result'] == 1) & 
                (result_df['Match'] == False)]['session_id'].to_list() )) 
         & (data_raw['chat_text'].str.contains("|".join(FederalDisclosure_regex_baselines), na=False, case=False)) 
        ][['session_id', 'sequence', 'chat_text', "vchproductsold"]].drop_duplicates().sort_values(['session_id', 'sequence']) #.shape[0]


In [ ]:
# form content


# form name


In [ ]:
data_raw[(data_raw['session_id'] == 'L17LEQKG6H2A77JNOPQIFJ6ST400HS19') & (~data_raw["chat_text"].isna()) 
         & (data_raw["entity_type"] == "agent") 
        ][['chat_text', 'sequence', 'entity_type', "vchproductsold", "VCHSTATE"]].drop_duplicates().sort_values(['sequence']) #.shape[0]


In [ ]:
data_raw[(data_raw["entity_type"] == "agent")
         & (~data_raw["chat_text"].isna())
         & (data_raw['chat_text'].str.contains("|".join(FederalDisclosure_regex_baselines_temp), na=False, case=False)) 
         #& (~data_raw['chat_text'].str.contains("|".join(FederalDisclosure_regex_baselines), na=False, case=False)) 
        ][['session_id', 'sequence', 'chat_text']].drop_duplicates().sort_values(['session_id', 'sequence']).shape[0]


In [ ]:
"I have reviewed the benefits and premium of the insurance policy and riders",
"I understand the impact that the premium for this coverage has on my paycheck income",
"I understand the impact that the total Aflac premium for this coverage",
"any other Aflac coverage has on my paycheck income and believe it to be appropriate for me",
"I have considered all of my existing health insurance coverage with Aflac",
"this additional coverage is appropriate for my insurance needs",
"I can contact Aflac insurance carriers to assist in evaluating the suitability of insurance coverage for me"

In [ ]:
    "benefits and premium of the insurance", "applying for and agree to the following",
    "understand the impact that the premium", "understand the impact of the premium", 
    "this additional coverage is appropriate", "assist in evaluating the suitability of insurance",
    "reviewed the benefits and premium", "my existing health insurance coverage",
    "applicant statements and agreements"
    "understands the impact that the premium", "understand the impact that the monthly premium",
    "reviewed the benefits and the premium of the insurance", "reviewed the benefits of the premium of the insurance",
    "benefits and the premium of the insurance",
    "quick disclosure", "suitability notice", 
    

In [ ]:
FederalDisclosure_regex_baselines_temp = [
    "disclosure"
    #"statements and agreements", "statement and agreements"
    #"suitability", "applicant statements", "disclosure statements and agreements",
    #"electronic delivery notice", "payment authorization agreement"
    #"form name", "form number", "mail"
]

data_raw[(data_raw['session_id'].isin(result_df[(result_df['kpi_result'] == 0) & (result_df['Actual'] == 1) & (result_df['Match'] == False)]['session_id'].to_list() )) 
         & (data_raw["entity_type"] == "agent")
         & (~data_raw["chat_text"].isna())
         & (data_raw['chat_text'].str.contains("|".join(FederalDisclosure_regex_baselines_temp), na=False, case=False)) 
         #& (~data_raw['chat_text'].str.contains("|".join(FederalDisclosure_regex_baselines), na=False, case=False)) 
        ][['session_id', 'sequence', 'chat_text']].drop_duplicates().sort_values(['session_id', 'sequence']) #.shape[0]


In [ ]:
data_raw[(data_raw['session_id'].isin(k[(k['engagement_flag'] == False)]['session_id'].to_list() )) ] \
        [['session_id', 'InteractionType']].drop_duplicates()['InteractionType'].value_counts()